In [1]:
import pandas as pd
import os
import pickle
from docx import Document
import re

In [2]:
path='../rawdata'

In [3]:

for key in os.listdir(path):
    print(key)

民事案件201811-201910 （渝北、江北）.xlsx
行政＋刑事16_17
案件列表201601-201712行政、刑事.xlsx
民事案件201801-10（渝北、江北）.xlsx
1.py
两法院民事案由分值.xlsx
.~lock.案件列表201601-201712行政、刑事 .xlsx#
民事18_19
两院法官人员信息.xlsx
案件列表201801-201910行政、刑事  .xlsx
18Data
.~lock.两院法官人员信息.xlsx#
数据清洗后
行政＋刑事18_19
.~lock.民事案件201811-201910 （渝北、江北）.xlsx#
.ipynb_checkpoints


## step1:4,5,6为案由计算

In [14]:
anyou=pd.read_excel(os.path.join(path,'两法院民事案由分值.xlsx')) 
anyou0=anyou[anyou['法院']=='重庆市江北区人民法院']
anyou1=anyou[anyou['法院']=='重庆市渝北区人民法院']

In [15]:
def anyou2id(anyou0):
    anyoudict0={}
    for index, row in anyou0.iterrows():
        anyoudict0[row[4]]=row[1]
    return anyoudict0

In [16]:
anyoudict0=anyou2id(anyou0)
anyoudict1=anyou2id(anyou1)

In [17]:
anyou.head(1)

,法院,分值,案由代码,上级代码,案由名称,案件类型,启用状态
0,重庆市江北区人民法院,6,9000,0,民事案由,民事,正常启用


## step2:去掉法官重名的情况，将人名与Id一一对应

In [18]:
judge=pd.read_excel(os.path.join(path,'两院法官人员信息.xlsx')) 

In [19]:
judge.head()

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
0,1,杨青,重庆市江北区人民法院,执行局,NaT,NaN,女,NaN,1996-01-22,NaN,2.0
1,2,凌文英,重庆市江北区人民法院,民事审判一庭,1985-12-20,审判员,女,NaN,1967-07-04,四级高级法官,1.0
2,3,张雪琪,重庆市江北区人民法院,执行局,NaT,NaN,女,NaN,1995-03-21,NaN,NaN
3,4,张敬,重庆市江北区人民法院,民事审判三庭,2001-07-04,审判员,女,副主任科员,1978-12-01,一级法官,1.0
4,5,汪涓,重庆市江北区人民法院,监察室,1986-07-01,审判员,女,副调研员,1964-03-06,四级高级法官,2.0


In [20]:
judge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 11 columns):
id        972 non-null int64
姓名        972 non-null object
法院        972 non-null object
部门        972 non-null object
入院日期      477 non-null datetime64[ns]
法律职务      476 non-null object
性别        972 non-null object
行政职务      315 non-null object
出生日期      972 non-null datetime64[ns]
等级        210 non-null object
是否员额法官    635 non-null float64
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 83.6+ KB


In [21]:
judge0=judge[judge['法院']=='重庆市江北区人民法院']
judge1=judge[judge['法院']=='重庆市渝北区人民法院']

In [22]:
#选出重复的名字：重庆市江北区人民法院：删除id==370
#渝北区人民法院：删除id==501,639,648,664,673,737,765,797,836
judge0[judge0['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
369,370,陈瑶,重庆市江北区人民法院,司法警察大队,NaT,NaN,男,NaN,1995-01-13,NaN,2.0
384,385,陈瑶,重庆市江北区人民法院,金融审判庭,NaT,NaN,女,NaN,1987-11-16,NaN,NaN


In [23]:
judge0.drop([369],inplace=True)
judge0[judge0['姓名'].duplicated(False)]

/home/huipu/.local/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官


In [24]:
judge1[judge1['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官
468,469,杨琼,重庆市渝北区人民法院,民事审判二庭,1994-04-01,庭长,女,庭长,1970-05-23,四级高级法官,1.0
500,501,何霜,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1989-10-18,NaN,NaN
638,639,陈果,重庆市渝北区人民法院,环境资源审判庭,NaT,NaN,女,NaN,1994-12-19,NaN,NaN
647,648,王婷婷,重庆市渝北区人民法院,综合办公室,2006-04-26,司法行政人员,女,其他,1983-01-27,NaN,2.0
659,660,陈果,重庆市渝北区人民法院,环境资源审判庭,2011-08-12,审判员,男,其他,1984-10-10,二级法官,1.0
663,664,马瑞,重庆市渝北区人民法院,民事审判三庭,2018-11-07,法官助理,女,其他,1993-12-17,NaN,NaN
672,673,何霜,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1994-08-18,NaN,2.0
736,737,郭鹏飞,重庆市渝北区人民法院,民事审判一庭,2018-11-08,法官助理,男,其他,1991-08-19,NaN,NaN
764,765,杨琼,重庆市渝北区人民法院,民事审判二庭,NaT,NaN,女,NaN,1991-10-07,NaN,NaN
796,797,马瑞,重庆市渝北区人民法院,民事审判三庭,NaT,NaN,女,NaN,1995-08-31,NaN,NaN


In [25]:
judge1.drop([500,638,647,663,672,736,764,796,835],inplace=True)
judge1[judge1['姓名'].duplicated(False)]

,id,姓名,法院,部门,入院日期,法律职务,性别,行政职务,出生日期,等级,是否员额法官


In [26]:
def judgeToId(judge1):
    judge2id={}
    # id2judge={}
    for index, row in judge1.iterrows():
        judge2id[row[1]]=row[0]
    #     id2judge[row[0]]=row[1]
    return judge2id

In [27]:
judge2id0=judgeToId(judge0)
judge2id1=judgeToId(judge1)

## step3:原被告数与主体类别

In [28]:
#原被告数与主体类别
#0表示自然人，1表示公司
#返回（自然人/公司，数量得分）
def plaintNum(s):
    numS=s.split(',')
    num=len(numS)
    flag=0#0表示自然人，1表示公司
    for key in numS:
        if len(key)>4:
            flag=1
    if num==1:
        return flag, 0
    elif num==2:
        return flag, 0.2
    else:
        return flag,0.5
plaintNum('兴业银行股份有限公司信用卡中心')

(1, 0)

## step4:适用程序

In [29]:
def proce(s):
    if s=='简易程序（小额诉讼）'or s=='简易程序(小额诉讼)':
        return 0
    if s=='简易程序(非小额诉讼)'or s=='简易程序（非小额诉讼）' or s=='简易':
        return 0.5
    if s=='普通程序'or s=='普通':
        return 1

## step5:标的金额

In [30]:
def money(m):
    if m=='\\N':
        m=0
    m=int(m)
    if m<=5000:
        return 0
    elif m<=30000:
        return 0.5
    else:
        return 1

# 案件打分，匹配法官（未在名单内的法官id为0）

201811-201910
江北区，共33221 条数据（包括判决和其他）,其中有5987条数据>=6
渝北区，共31389 条数据（包括判决和其他）,其中有10636条数据>=6
201801-201810   
江北区，共23209 条数据（包括判决和其他）,其中有3709条数据>=6
渝北区，共24117 条数据（包括判决和其他）,其中有8646条数据>=6

## step6:读取excel数据

In [31]:
root_pre='民事2018.01-2018.10.xlsx'
data_pre=pd.read_excel(os.path.join(path,'数据清洗后',root_pre))   

In [32]:
data_pre0=data_pre[data_pre['法院']=='重庆市江北区人民法院']
data_pre1=data_pre[data_pre['法院']=='重庆市渝北区人民法院']

In [33]:
root='民事2018.11-2019.10.xlsx'
data=pd.read_excel(os.path.join(path,'数据清洗后',root))   

In [34]:
data0=data[data['法院']=='重庆市江北区人民法院']
data1=data[data['法院']=='重庆市渝北区人民法院']

In [35]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22867 entries, 0 to 22866
Data columns (total 24 columns):
案件标识           22867 non-null object
法院             22867 non-null object
案号             22867 non-null object
立案案由           22867 non-null object
原告             22867 non-null object
被告             22867 non-null object
承办部门           22867 non-null object
承办人            22867 non-null object
申请标的           22867 non-null object
适用程序           22867 non-null object
收案日期           22867 non-null datetime64[ns]
立案日期           22867 non-null datetime64[ns]
移送日期           22867 non-null object
签收日期           22867 non-null datetime64[ns]
分案日期           22867 non-null object
报结日期           22867 non-null object
结案日期           22867 non-null datetime64[ns]
结案方式           22867 non-null object
裁判文书           22867 non-null object
法定审理天数         22867 non-null int64
实际审理天数         22867 non-null int64
计算结果           22867 non-null int64
Unnamed: 22    0 non-null float64
差值判断          

### 案件打分，输出格式：{'案号'：{'案由','案件得分','实际时间','法官id'}}

In [36]:
def score(data0,anyoudict0,judge2id0):
    datadict={}
    number=0
    for index, row in data0.iterrows():
        try:
            content={}
            content['案由']=row[3]
            a,b=plaintNum(row[4])
            c,d=plaintNum(row[5])
            num=anyoudict0[row[3]]
            num=num+b+d+(a+c)/2
            num=num+money(row[8])
            num=num+proce(row[9])
            content['案件得分']=num
            content['实际时间']=int(row[21])
            if num>=6:
                number=number+1
            if row[7] in judge2id0:
                content['法官id']=judge2id0[row[7]]
            else:
                content['法官id']=0
            content['适用程序']=row[9]
    #         content['案由得分']=anyoudict0(row[3])
    #         content['原告得分']=b
    #         content['被告得分']=d
    #         content['标的金额']=money(row[8])
    #         content['适用程序']=proce(row[9])
    #         content['主体类别']=(a+c)/2
            datadict[row[2]]=content
        except:
            continue
    print(number)
    return datadict

In [37]:
datadict1=score(data1,anyoudict1,judge2id1)

2943


In [38]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11567 entries, 3 to 22866
Data columns (total 24 columns):
案件标识           11567 non-null object
法院             11567 non-null object
案号             11567 non-null object
立案案由           11567 non-null object
原告             11567 non-null object
被告             11567 non-null object
承办部门           11567 non-null object
承办人            11567 non-null object
申请标的           11567 non-null object
适用程序           11567 non-null object
收案日期           11567 non-null datetime64[ns]
立案日期           11567 non-null datetime64[ns]
移送日期           11567 non-null object
签收日期           11567 non-null datetime64[ns]
分案日期           11567 non-null object
报结日期           11567 non-null object
结案日期           11567 non-null datetime64[ns]
结案方式           11567 non-null object
裁判文书           11567 non-null object
法定审理天数         11567 non-null int64
实际审理天数         11567 non-null int64
计算结果           11567 non-null int64
Unnamed: 22    0 non-null float64
差值判断          

In [39]:
datadict1['(2018)渝0112民初30233号']

{'案由': '房屋租赁合同纠纷', '案件得分': 5.5, '实际时间': 82, '法官id': 464, '适用程序': '简易程序（非小额诉讼）'}

In [40]:
datadict0=score(data0,anyoudict0,judge2id0)

860


In [41]:
predict0=score(data_pre0,anyoudict0,judge2id0)

700


In [42]:
predict1=score(data_pre1,anyoudict1,judge2id1)

2744


In [43]:
len(datadict1)

11567

In [44]:
len(datadict0)

11300

In [45]:
print(len(predict0),len(predict1))

5021 10006


合并2018-2019年的案件,datadict0:2018.11-2019.10;predict0:2018.01-2018.10

In [46]:
datadict0.update(datadict1)
predict0.update(predict1)

In [47]:
print(len(datadict0),len(predict0),len(datadict0)+len(predict0))

22867 15027 37894


## 读取18-19年以判决结案的案号与文书内容（有判决书的案件,共48441条）

### :将案件标识与案号关联:keyDict

In [48]:
#将案件标识与案号关联:keyDict
#root为文书存放目录名，path为保存keyDict的文件名
def signDict(path1,rootw,pathn):
    keyDict={}#将案件标识与案号关联
    table=[]#以表格形式存在的文书
    notFindDoc = []#不是以docx结尾的文书（格式转换未成功的文书）
    if os.path.exists(os.path.join(rootw,pathn)):
        read_file = open(os.path.join(rootw,pathn), 'rb')
        keyDict = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'table.pkl'), 'rb')
        table = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'notFindDoc.pkl'), 'rb')
        notFindDoc = pickle.load(read_file)
        read_file.close()
    else:
        path2=os.path.join(path1,'民事18_19')
        for f in os.listdir(path2):
            root2=os.path.join(path2, f)
            for key_ in os.listdir(root2) :
                key = os.path.join(root2,key_)
                try:
                    if key.endswith(".docx"):
                        document = Document(key)
                        paras = list()
                        for p in document.paragraphs:
                            if p.text.strip() != "":
                                paras.append(p.text.strip())
                        # if(len(paras)==0):
                        #     tables = document.tables
                        #     for t in tables:
                        #         for r in t.rows:
                        #             for c in r.cells:
                        #                 for p in c.paragraphs:
                        #                     if p.text.strip() != "":
                        #                         paras.append(p.text.strip())
                        keyDict[key_.split('.')[0]]=paras[2]
                    else:
                        notFindDoc.append(key)
                except:
                    table.append(key)
        output = open(os.path.join(rootw,pathn), 'wb')
        pickle.dump(keyDict, output,True)
        output.close()
        output = open(os.path.join(rootw,'table.pkl'), 'wb')
        pickle.dump(table, output,True)
        output.close()
        output = open(os.path.join(rootw,'notFindDoc.pkl'), 'wb')
        pickle.dump(notFindDoc, output,True)
        output.close()
    print('keyDict',len(keyDict),'table',len(table),'notFindDoc',len(notFindDoc))
    return keyDict,table,notFindDoc

In [49]:
import os
path='../rawdata'
rootw=os.path.join(path,'数据清洗后','18Data')
pathn='keyDict.pkl'

In [50]:
keyDict,table,notFindDoc=signDict(path,rootw,pathn)

keyDict 42609 table 3785 notFindDoc 1950


In [51]:
keyDict;
#{'M1D_7cc626c2-50c2-4921-a27e-ec10428feaa8': '（2019）渝0112民初9935号','M1D_98cb7509-1656-4804-a762-0e21f1140a0d': '渝0112民初19754号',

### 2,提取文书内容

### keyDict key与value置换（keyDcitRe），选出在excel和文书都存在的案件==>keyDcitReplace
datadictFinal为excel2018-2019内的案号与案件标识

In [52]:
#datadictFinal为excel2018-2019内的案号与案件标识
datadictFinal={}
datadictFinal.update(datadict0)
datadictFinal.update(predict0)

In [53]:
#data_pre.head()
# data.head(1)

In [153]:
datadictFinal['(2019)渝0112民初9935号']

{'案由': '劳动争议', '案件得分': 7.0, '实际时间': 88, '法官id': 484, '适用程序': '简易程序(小额诉讼)'}

In [55]:
def keyReplace0(keyDict):
    keyDcitRe={}
    for key in keyDict:
        keyDcitRe[keyDict[key]]=key
    return keyDcitRe
def keyReplace(datadictFinal,keyDict):
    keyDcitRe=keyReplace0(keyDict)
    keyDcitReplace={}
    biaoshiKey={}
    notExcel={}
    for key in keyDcitRe:
        temp = keyDcitRe[key]
        key=key.replace('（','(')
        key=key.replace('）',')')
        if key in datadictFinal:
            keyDcitReplace[key] = temp
            biaoshiKey[temp]=key
        else:
            notExcel[key]=temp
    return keyDcitReplace,biaoshiKey,notExcel

In [56]:
def getKeyDictReplace(datadictFinal,keyDict):
    keyDcitReplace,biaoshiKey,notExcel=keyReplace(datadictFinal,keyDict)
    return keyDcitReplace,biaoshiKey,notExcel

In [57]:
keyDcitReplace,biaoshiKey,notExcel=getKeyDictReplace(datadictFinal,keyDict)

In [58]:
len(keyDcitReplace)#
#{'(2019)渝0112民初9935号': 'M1D_7cc626c2-50c2-4921-a27e-ec10428feaa8',
#  '(2019)渝0112民初33199号': 'M1D_0684a933-2245-486b-9020-df4d4182158f',

31501

### 统计法官工作量

In [ ]:
def work(judge2id0,judge2id1):
    data.drop(['Unnamed: 22'],axis=1,inplace=True)
    excel_data=data.append(data_pre)
    excel_data2= pd.DataFrame(columns=excel_data.columns.values )#两方都在的案件信息
    drop_excel=[]#统计不在文书集中的案件
    kk=0
    for index, row in excel_data.iterrows():

        if row[2] not in keyDcitReplace:
            drop_excel.append(index);
        else:
            excel_data2.loc[kk] = row;
            kk=kk+1
    print(len(excel_data)-len(drop_excel)==len(excel_data2))#判断是否正确
    excel_data2_0=excel_data2[excel_data2['法院']=='重庆市江北区人民法院']
    excel_data2_0['法官id']=excel_data2_0['承办人'].apply(lambda x:judge2id0[x] if x in judge2id0 else 0)
    excel_data2_1=excel_data2[excel_data2['法院']=='重庆市渝北区人民法院']
    excel_data2_1['法官id']=excel_data2_0['承办人'].apply(lambda x:judge2id1[x] if x in judge2id1 else 0)
    excel_data2_p=excel_data2_0.append(excel_data2_1);
    excel_data2_p['工作量']=0
    # temp['立案日期'].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"));datetime转字符串
    for index,row in excel_data2_p.iterrows():
    #     print(row[11])
        temp=excel_data2_p[excel_data2_p['法官id']==row[23]]
        excel_data2_p.loc[index,'工作量']=len(temp[(temp['立案日期']<=pd.to_datetime(row[11])) & (temp['结案日期']>pd.to_datetime(row[11]))])
    return excel_data2_p

In [59]:
data.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,Unnamed: 22,差值判断
0,00055d19-b4a4-4672-9469-dc088ca360c7,重庆市江北区人民法院,(2018)渝0105民初34609号,信用卡纠纷,兴业银行股份有限公司信用卡中心,刘祖康,金融审判庭,毛亚男,53984,简易程序（非小额诉讼）,...,2018-10-22 00:00:00,2018-12-13 00:00:00,2018-12-14,判决,民事判决书（当事人对案件事实没有争议的用）,92,57,57,NaN,0


In [60]:
data.drop(['Unnamed: 22'],axis=1,inplace=True)

In [61]:
data_pre.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,签收日期,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断
0,00077602-304a-4a5c-a069-96e2ae933f11,重庆市江北区人民法院,(2018)渝0105民初31680号,信用卡纠纷,兴业银行股份有限公司信用卡中心,伯春艳,金融审判庭,郭雪妍,50838.4,简易程序（非小额诉讼）,...,2018-08-01,2018-08-01,2018-09-10 00:00:00,2018-09-10,判决,民事判决书(一审简易程序用),92,45,45,0


In [63]:
excel_data=data.append(data_pre)

In [64]:
len(excel_data)

37894

In [65]:
excel_data.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,签收日期,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断
0,00055d19-b4a4-4672-9469-dc088ca360c7,重庆市江北区人民法院,(2018)渝0105民初34609号,信用卡纠纷,兴业银行股份有限公司信用卡中心,刘祖康,金融审判庭,毛亚男,53984,简易程序（非小额诉讼）,...,2018-10-22,2018-10-22 00:00:00,2018-12-13 00:00:00,2018-12-14,判决,民事判决书（当事人对案件事实没有争议的用）,92,57,57,0


In [66]:
excel_data2= pd.DataFrame(columns=excel_data.columns.values )

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,签收日期,分案日期,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断


In [75]:
# excel_data2= pd.DataFrame(excel_data.)
drop_excel=[]
kk=0
for index, row in excel_data.iterrows():
    
    if row[2] not in keyDcitReplace:
        drop_excel.append(index);
    else:
        excel_data2.loc[kk] = row;
        kk=kk+1
        

In [154]:
excel_data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31501 entries, 0 to 31500
Data columns (total 23 columns):
案件标识      31501 non-null object
法院        31501 non-null object
案号        31501 non-null object
立案案由      31501 non-null object
原告        31501 non-null object
被告        31501 non-null object
承办部门      31501 non-null object
承办人       31501 non-null object
申请标的      31501 non-null object
适用程序      31501 non-null object
收案日期      31501 non-null datetime64[ns]
立案日期      31501 non-null datetime64[ns]
移送日期      31501 non-null object
签收日期      31501 non-null datetime64[ns]
分案日期      31501 non-null object
报结日期      31501 non-null object
结案日期      31501 non-null datetime64[ns]
结案方式      31501 non-null object
裁判文书      31501 non-null object
法定审理天数    31501 non-null int64
实际审理天数    31501 non-null int64
计算结果      31501 non-null int64
差值判断      31501 non-null int64
dtypes: datetime64[ns](4), int64(4), object(15)
memory usage: 7.0+ MB


In [78]:
len(drop_excel)

6393

In [79]:
drop_excel;

In [80]:
len(excel_data)-len(drop_excel)

31501

In [90]:
excel_data2_0=excel_data2[excel_data2['法院']=='重庆市江北区人民法院']
excel_data2_0['法官id']=excel_data2_0['承办人'].apply(lambda x:judge2id0[x] if x in judge2id0 else 0)
excel_data2_1=excel_data2[excel_data2['法院']=='重庆市渝北区人民法院']
excel_data2_1['法官id']=excel_data2_0['承办人'].apply(lambda x:judge2id1[x] if x in judge2id1 else 0)
excel_data2_p=excel_data2_0.append(excel_data2_1);

/home/huipu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/huipu/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [140]:
excel_data2_p['工作量']=0
# temp['立案日期'].apply(lambda x:x.strftime("%Y-%m-%d %H:%M:%S"));datetime转字符串
for index,row in excel_data2_p.iterrows():
#     print(row[11])
    temp=excel_data2_p[excel_data2_p['法官id']==row[23]]
    excel_data2_p.loc[index,'工作量']=len(temp[(temp['立案日期']<=pd.to_datetime(row[11])) & (temp['结案日期']>pd.to_datetime(row[11]))])


In [141]:
excel_data2_p.head(1)

,案件标识,法院,案号,立案案由,原告,被告,承办部门,承办人,申请标的,适用程序,...,报结日期,结案日期,结案方式,裁判文书,法定审理天数,实际审理天数,计算结果,差值判断,法官id,工作量
0,0012e26f-a00c-4540-839d-b82aca6aeb18,重庆市江北区人民法院,(2018)渝0105民初31379号,金融借款合同纠纷,中国民生银行股份有限公司重庆分行,"陈剑芳,何文任",金融审判庭,张静,819018,普通程序,...,2018-12-21 00:00:00,2018-12-21,判决,民事判决书（一审普通程序用—全面版）,184,94,155,0,1,5151


In [143]:
excel_data2_p.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31501 entries, 0 to 31500
Data columns (total 25 columns):
案件标识      31501 non-null object
法院        31501 non-null object
案号        31501 non-null object
立案案由      31501 non-null object
原告        31501 non-null object
被告        31501 non-null object
承办部门      31501 non-null object
承办人       31501 non-null object
申请标的      31501 non-null object
适用程序      31501 non-null object
收案日期      31501 non-null datetime64[ns]
立案日期      31501 non-null datetime64[ns]
移送日期      31501 non-null object
签收日期      31501 non-null datetime64[ns]
分案日期      31501 non-null object
报结日期      31501 non-null object
结案日期      31501 non-null datetime64[ns]
结案方式      31501 non-null object
裁判文书      31501 non-null object
法定审理天数    31501 non-null int64
实际审理天数    31501 non-null int64
计算结果      31501 non-null int64
差值判断      31501 non-null int64
法官id      31501 non-null int64
工作量       31501 non-null int64
dtypes: datetime64[ns](4), int64(6), object(15)
memory usage: 7.5

In [157]:
rootw=os.path.join(path,'数据清洗后','18Data')
excel_data2_p.to_csv(os.path.join(rootw,'已计算工作量.csv')) 

### 提取描述

In [145]:
import json
import os
from docx import Document
import re
import pickle
#将案件标识与案号关联:keyDict
#root为文书存放目录名，path为保存keyDict的文件名
def signDict(rootw,pathn):
    #保存三个文件：keyDict,table,notFindDoc
    keyDict={}#将案件标识与案号关联
    table=[]#以表格形式存在的文书
    notFindDoc = []#不是以docx结尾的文书（格式转换未成功的文书）
    if os.path.exists(os.path.join(rootw,pathn)):
        read_file = open(os.path.join(rootw,pathn), 'rb')
        keyDict = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'table.pkl'), 'rb')
        table = pickle.load(read_file)
        read_file.close()
        read_file = open(os.path.join(rootw,'notFindDoc.pkl'), 'rb')
        notFindDoc = pickle.load(read_file)
        read_file.close()
    else:
        for f in os.listdir(rootw):
            root2=os.path.join(rootw, f)
            for key_ in os.listdir(root2) :
                key = os.path.join(root2,key_)
                try:
                    if key.endswith(".docx"):
                        document = Document(key)
                        paras = list()
                        for p in document.paragraphs:
                            if p.text.strip() != "":
                                paras.append(p.text.strip())
                        # if(len(paras)==0):
                        #     tables = document.tables
                        #     for t in tables:
                        #         for r in t.rows:
                        #             for c in r.cells:
                        #                 for p in c.paragraphs:
                        #                     if p.text.strip() != "":
                        #                         paras.append(p.text.strip())
                        keyDict[key_.split('.')[0]]=paras[2]
                    else:
                        notFindDoc.append(key)
                except:
                    table.append(key)
        output = open(os.path.join(rootw,pathn), 'wb')
        pickle.dump(keyDict, output,True)
        output.close()
        output = open(os.path.join(rootw,'table.pkl'), 'wb')
        pickle.dump(table, output,True)
        output.close()
        output = open(os.path.join(rootw,'notFindDoc.pkl'), 'wb')
        pickle.dump(notFindDoc, output,True)
        output.close()
    print('keyDict',len(keyDict),'table',len(table),'notFindDoc',len(notFindDoc))
    return keyDict,table,notFindDoc


# 读取段落文字
def readPara(doc):  # doc为待读取的文书路径
    document = Document(doc)
    paras=list()
    data=list()
    dang=""
    for p in document.paragraphs:
        if p.text.strip() !="" :
            paras.append(p.text.strip())
            # paras.insert(0,zhuti)
    for i in range(len(paras)):
        if '原告'not in paras[i] and '被告' not in paras[i]:
            data.append(paras[i])
        else:
            paras=paras[i:len(paras)]
            break
    p1 = re.compile('原告.*案')
    # p2=re.compile('被告.*：')
    for i in range(len(paras)):
        if len(p1.findall(paras[i]))==0:
            paras[i]=paras[i].replace("\n", "")
            dang=dang+paras[i].strip()
        else:
            paras=paras[i:len(paras)]
            break
    dang2 = re.split('被告', dang[10:], 1)
    dang2[0] = dang[0:10] + dang2[0]
    data.extend(dang2)
    data.extend(paras)
    return data


#  step 1 将doc处理成字典格式--记录读取的内容到filename
def filter0(root,filename,keyDict,rootw):#root='../rawdata/民事18_19'  rootw='../rawdata/18Data'
    #共保存两个文件：filename,'notread.txt'
    notread=[]
    data=[]#目的是为了直观
    if os.path.exists(os.path.join(rootw,filename)):
        print(filename+'exists')
        with open(os.path.join(rootw,filename),'r',encoding='utf-8')as f:
            for line in f:
                d=json.loads(line)#转成字典格式
                data.append(d)
    else:
        p1 = re.compile('(.*)事实.*?理由(.*)')
        p2=re.compile('依照(.*)规定')
        for f in os.listdir(root):
            root2=os.path.join(root, f)
            for key in os.listdir(root2) :
                key2 = os.path.join(root2,key)
                datatemp = dict()
#                 temp = key.split('.')[0]
                if key.split('.')[0] in keyDict:
                    try:
                        paras = readPara(key2)
                        if len(paras) == 0:
                            notread.append(key)
                        if len(paras) > 0:
                            datatemp['key']=keyDict[key.split('.')[0]]
                            datatemp['原告'] = paras[3]
                            datatemp['被告'] = paras[4]
                            datatemp['案由+程序'] = paras[5]

                            if '请求' in paras[6]:
                                temp = p1.findall(paras[6])
                                if len(temp) > 0:
                                    temp = temp[0]
                                    if len(temp[1]) > 5:
                                        datatemp['诉讼请求'] = temp[0]
                                        datatemp['事实理由'] = temp[1]
                                    else:
                                        datatemp['诉讼请求'] = paras[6]
                                        datatemp['事实理由'] = paras[7]
                                else:
                                    datatemp['诉讼请求'] = paras[6]
                                    datatemp['事实理由'] = paras[7]
                                if('经审理' in paras[9]):
                                    datatemp['事实理由'] = paras[9]
                                elif('经审理' in paras[8]):
                                    datatemp['事实理由'] = paras[8]
                                temp=p2.findall(paras[12])
                                if(len(temp)>0):
                                    datatemp['适用法条'] = temp[0]
                                else:
                                    datatemp['适用法条'] = ''
                                data.append(datatemp)

                            else:
                                notread.append(key)
                    except:
                        notread.append(key)
#                 else:
#                     print(key.split('.')[0])
                    # error.append(key)
#                 break
#             break
        print('读取失败的文书',len(notread))
        #记录读取的内容
        with open(os.path.join(rootw,filename), 'w', encoding='utf-8')as f2:
            f2.truncate()  # 清空文件内容
            for key in data:
                json.dump(key, f2, ensure_ascii=False)
                f2.write('\n')

        #记录未读的路径
        with open(os.path.join(rootw,'notread.txt'), "w") as fw:
            fw.truncate()
            for i in range(len(notread)):
                print(str(notread[i]) , file=fw)
    print('记录的文书量：',len(data))
    return data

# step 2 将以列表形式存在的json变为一个json存储-->保存在fileE中
def jsonOne(rootw,fileB,fileE):
    if os.path.exists(os.path.join(rootw,fileE)):
        read_file = open(os.path.join(rootw,fileE), 'rb')
        data = pickle.load(read_file)
        read_file.close()
    else:
        with open(os.path.join(rootw,fileB), 'r', encoding='utf-8')as f:
            data = dict()
            for line in f:
                d = json.loads(line)  # 转成字典格式
                k=d['key']
                data[k]= -1
                del d["key"]
                data[k]=d
        output = open(os.path.join(rootw,fileE), 'wb')
        pickle.dump(data, output,True)
        output.close()
    return data

# # if __name__ == '__main__':
#     path = '../rawdata/数据清洗后'
#     rootw = os.path.join(path, '18Data')
# #     pathn = 'keyDict.pkl'
# #     keyDict,table,notFindDoc=signDict(rootw,pathn)
#     data0=filter0(rootw, 'dataRead.json', keyDcitReplace)
#     data=jsonOne('dataRead.json', 'data.json')

In [146]:
root='../rawdata/民事18_19'
rootw = '../rawdata/数据清洗后/18Data'
# keyDict,table,notFindDoc=signDict(path,rootw,pathn)
data0=filter0(root, 'dataRead.json', keyDict,rootw)

dataRead.jsonexists
记录的文书量： 41644


In [147]:
data=jsonOne(rootw,'dataRead.json', 'datajsonOne.json')

### 4, 将同在文书集与excel表中的案件进行提取，输出格式：{案号：{原告，被告，案由+程序,诉讼请求，事实理由，适用法条}，}

In [83]:
#将同在文书集与excel表中的案件进行提取，输出格式：{案号：{原告，被告，案由+程序,诉讼请求，事实理由，适用法条}，}
def getContent(root,dataFile,filename,biaoshiKey):#dataFile='content.json',filename='contentList.json'
    if os.path.exists(os.path.join('../rawdata',dataFile)):
        content=jsonOne(filename, dataFile)
    else:
        path = '../rawdata'
        rootw = os.path.join(path, root)
        data0=filter0(rootw, filename, biaoshiKey)
        content=jsonOne(filename, dataFile)
        return content

In [84]:
# dataFile='content.json'
# filename='contentList.json'
# content=getContent('2',dataFile,filename,biaoshiKey);

### 5,获取最终数据

In [85]:
-

In [86]:
# data=combineInfo(content,datadictFinal,'data.json')

### 6,合并所有信息

In [87]:

 #将excel表中的数据处理，输出格式：{'案号'：{'案由','案件得分','实际时间','法官id'}}
def getDataFinal(anyoudict1,judge2id1,anyoudict0,judge2id0):
    root='民事案件201811-201910 （渝北、江北）.xlsx'
    data=pd.read_excel(os.path.join(path,root))   
    data0=data[data['法院']=='重庆市江北区人民法院']
    data1=data[data['法院']=='重庆市渝北区人民法院']
    root_pre='民事案件201801-10（渝北、江北）.xlsx'
    data_pre=pd.read_excel(os.path.join(path,root_pre))   
    data_pre0=data_pre[data_pre['法院']=='重庆市江北区人民法院']
    data_pre1=data_pre[data_pre['法院']=='重庆市渝北区人民法院']
    datadict1=score(data1,anyoudict1,judge2id1)
    datadict0=score(data0,anyoudict0,judge2id0)
    predict0=score(data_pre0,anyoudict0,judge2id0)
    predict1=score(data_pre1,anyoudict1,judge2id1)
    DataFinal={}
    DataFinal.update(datadict0)
    DataFinal.update(datadict1)
    DataFinal.update(predict0)
    DataFinal.update(predict1)
    return DataFinal

#将文书集中的案件标识与案号统一
def getKeyDict(rootw,pathn):
#     path='../rawdata'
#     rootw=os.path.join(path,'1')
#     pathn='keyDict.pkl'
    keyDict,table,notFindDoc=signDict(rootw,pathn)
    return keyDict

#将同在文书集与excel表中的案号提取出来，输出格式：{'案号': '案件标识',
def getKeyDictReplace(datadictFinal,keyDict):
    keyDcitReplace,biaoshiKey,notExcel=keyReplace(datadictFinal,keyDict)
    return keyDcitReplace,biaoshiKey,notExcel

#将同在文书集与excel表中的案件进行提取，输出格式：{案号：{原告，被告，案由+程序,诉讼请求，事实理由，适用法条}，}
def getContent(root,dataFile,filename,keyDcitReplace):#dataFile='content.json',filename='contentList.json'
    if os.path.exists(os.path.join('../rawdata',dataFile)):
        content=jsonOne(filename, dataFile)
    else:
        path = '../rawdata'
        rootw = os.path.join(path, root)
        data0=filter0(rootw, filename, keyDcitReplace)
        content=jsonOne(filename, dataFile)
        return content

# 获取最终数据
def combineInfo(content,datadictFinal,filename):
    '''
    content:保证在excel和文书集共同存在：{案号：{原告，被告，案由+程序,诉讼请求，事实理由，适用法条}，}
    :datadictFinal: 处理后的案件列表:{案号:{'案由': '信用卡纠纷', '案件得分': 5.0, '实际时间': 57, '法官id': 51}，}
    :filename:将数据return之前，保存在filename文件中
    :return: 列表与json文件的组合：{'content': {'原告', '被告', '案由+程序', '诉讼请求', '事实理由'},
                                     'detail': {'案由', '案件得分'，'实际时间'，'法官id'}}
    '''
    if os.path.exists(os.path.join('../rawdata',filename)):
        read_file = open(os.path.join('../rawdata',filename), 'rb')
        data = pickle.load(read_file)
        read_file.close()
    else:
        high=0
        low=0
        data={}
        for key in content:
            if key in datadictFinal:
                temp={}
                temp['content']=content[key]
                temp['detail']=datadictFinal[key]
                if datadictFinal[key]['案件得分']>=6:
                    high=high+1
                else:
                    low=low+1
                data[key]=temp
        output = open(os.path.join('../rawdata',filename), 'wb')
        pickle.dump(data, output,True)
        output.close()
        print('>=6',high,'<6',low)
    return data

In [90]:
# data['(2018)渝0105民初31174号']

In [72]:
rootw = os.path.join(path, '2')
pathn = 'keyDict.pkl'
dataFile='content.json'
filename='contentList.json'

In [73]:
keyDict=getKeyDict(rootw,pathn)

keyDict 42609 table 1 notFindDoc 0


In [79]:
keyDcitReplace0,biaoshiKey0,notExcel0=getKeyDictReplace(datadictFinal,keyDict)

In [80]:
print(len(keyDcitReplace0),len(biaoshiKey0),len(notExcel0))

41997 41999 487


In [81]:
content=getContent('2',dataFile,filename,biaoshiKey0);

读取失败的文书 938
记录的文书量： 41062


In [83]:
len(content)

41060

In [84]:
content['(2019)渝0112民初9935号']

{'原告': '原告：张永江,男,土家族,1990年12月17日出生, 住重庆市黔江区濯水镇双龙村5组,公民身份号码500239199012175752。委托诉讼代理人：张高易，重庆市渝北区两路法律服务所法律工作者。',
 '被告': ':重庆千彩万家装饰工程有限公司,住所地重庆市渝北区双龙湖街道兰馨大道18号附10号御城华府22幢2-商场。 统一社会信用代码91500112MA5YTFW34A。法定代表人: 刘冬。',
 '案由+程序': '原告张永江与被告重庆千彩万家装饰工程有限公司劳动争议一案，本院受理后，依法由审判员徐振中适用简易程序中的小额诉讼程序公开开庭进行了审理，原告委托诉讼代理人张高易到庭参加诉讼，被告经本院合法传唤，无正当理由未到庭参加诉讼，本院依法缺席审理。本案现已审理终结。',
 '诉讼请求': '原告张永江向本院提出诉讼请求：被告支付原告工资 16 948元。',
 '事实理由': '经审理查明：在原告举示的《关于千彩万家装饰2018年度工地结款协议书》（签订日期为2019年1月18日）上有以下内容：千彩万家装饰原项目经理钟磊在公司承包的在建工地，公司已按时发放款项给钟磊，由于钟磊未能及时发放给工人，现经公司与工人友好沟通协商。决定此款项由公司从钟磊剩余款项中扣除出来，直接发放给工人。合计欠款金额为16 948元，经双方友好协商，此项欠款商定在2019年1月25号前付8474元，剩余款项8474元定在2019年4月25号前付清。公司盖章处加盖重庆千彩万家装饰工程有限公司公章，工人处有张永江签署名字。',
 '适用法条': ''}

In [85]:
data=combineInfo(content,datadictFinal,'data.pkl')

In [86]:
len(data)

41060

In [87]:
data['(2019)渝0112民初4074号']

{'content': {'原告': '原告：肖瑾，女，1983年1月15日出生，汉族，住重庆市渝北区云竹路6号联发瞰青1期，公民身份号码510202198301157328。',
  '被告': '：罗健，男，1981年1月28日出生，汉族，住安徽省宁国市西津办事处大村吴冲下组16号，公民身份号码342524198101282217。',
  '案由+程序': '原告肖瑾诉被告罗健抚养费纠纷一案，2019年2月13日由本院立案受理后，依法由审判员任韵霖适用简易程序于2019年3月11日公开开庭进行了审理。原告肖瑾、被告罗健到庭参加了诉讼，本案现已审理终结。',
  '诉讼请求': '原告肖瑾向本院提出诉讼请求：请求法院判令1.被告支付婚生女罗钿雅2018.7至2019.2期间的生活费15 500元、医药保险费550元，共计16 050元；2.被告支付违约金2000元。本案诉讼费由被告承担。',
  '事实理由': '本院经审理认定如下事实：原告肖瑾与被告罗健原系夫妻关系。二人于2017年2月8日生育女儿罗钿雅。2018年4月26日，肖瑾与罗健在重庆市渝北区民政局协议离婚，并约定罗钿雅由肖瑾抚养，被告罗健每月给付婚生女生活费3000元直至女儿能独立生活为止，另托养费、教育费、医疗费、保险费由被告全部负责（如原告工作后，被告支付罗钿雅每月生活费2000元，女儿产生其他托养费、教育费或医疗费用：被告承担70%，原告承担30%，如需产生上述费用，原告必须提前和被告协商后而定）。若任何一方不按本协议约定期限和足额支付生活费，应按未支付款项30%承担违约金支付给对方，并按未付款项千分之二计日息，按日累计，直到付清为止。生活费、托养费、教育费、医疗费、保险费凭相关票据在十五日内均由被告按照前序条款约定承担，如逾期未支付上述费用，被告将承担违约金30%，并按未付款项千分之二计日息，按日累计，直到付清为止。原告与被告离婚后，罗钿雅跟随原告肖瑾居住生活。原告肖瑾现无工作，处于失业状态。',
  '适用法条': '《中华人民共和国婚姻法》第二十一条第一款、第二款、第三十七条，《最高人民法院关于适用<中华人民共和国婚姻法>若干问题的解释（一）》第二十一条和《中华人民共和国民事诉讼法》第六十四条、第一百五十二条之'},
 'detail': {'案由': '抚养费纠纷',
  '案件得分'

In [88]:
data['(2019)渝0112民初9935号']

{'content': {'原告': '原告：张永江,男,土家族,1990年12月17日出生, 住重庆市黔江区濯水镇双龙村5组,公民身份号码500239199012175752。委托诉讼代理人：张高易，重庆市渝北区两路法律服务所法律工作者。',
  '被告': ':重庆千彩万家装饰工程有限公司,住所地重庆市渝北区双龙湖街道兰馨大道18号附10号御城华府22幢2-商场。 统一社会信用代码91500112MA5YTFW34A。法定代表人: 刘冬。',
  '案由+程序': '原告张永江与被告重庆千彩万家装饰工程有限公司劳动争议一案，本院受理后，依法由审判员徐振中适用简易程序中的小额诉讼程序公开开庭进行了审理，原告委托诉讼代理人张高易到庭参加诉讼，被告经本院合法传唤，无正当理由未到庭参加诉讼，本院依法缺席审理。本案现已审理终结。',
  '诉讼请求': '原告张永江向本院提出诉讼请求：被告支付原告工资 16 948元。',
  '事实理由': '经审理查明：在原告举示的《关于千彩万家装饰2018年度工地结款协议书》（签订日期为2019年1月18日）上有以下内容：千彩万家装饰原项目经理钟磊在公司承包的在建工地，公司已按时发放款项给钟磊，由于钟磊未能及时发放给工人，现经公司与工人友好沟通协商。决定此款项由公司从钟磊剩余款项中扣除出来，直接发放给工人。合计欠款金额为16 948元，经双方友好协商，此项欠款商定在2019年1月25号前付8474元，剩余款项8474元定在2019年4月25号前付清。公司盖章处加盖重庆千彩万家装饰工程有限公司公章，工人处有张永江签署名字。',
  '适用法条': ''},
 'detail': {'案由': '劳动争议',
  '案件得分': 7.0,
  '实际时间': 88,
  '法官id': 484,
  '适用程序': '简易程序(小额诉讼)'}}

In [89]:
datadictFinal['(2019)渝0112民初9935号']

{'案由': '劳动争议', '案件得分': 7.0, '实际时间': 88, '法官id': 484, '适用程序': '简易程序(小额诉讼)'}

In [226]:
keyDcitReplace0['(2019)渝0112民初9935号']

'M1D_7cc626c2-50c2-4921-a27e-ec10428feaa8'

In [235]:
len(data['(2019)渝0112民初4074号']['content']['适用法条'])

99

In [237]:
fatiao=0
notFatiao=0
for key in data:
    if len(data[key]['content']['适用法条'])>0:
        fatiao=fatiao+1
    else:
        notFatiao=notFatiao+1
print('法条识别出的案件数:',fatiao,'法条没有识别出的案件数：',notFatiao)

法条识别出的案件数: 4292 法条没有识别出的案件数： 36768


In [240]:
os.path.join(path, '2')

'../rawdata/2'

## 求某列字符串长度

In [263]:
test = {'Science':[1,2,3],'Art':[4,5,6]}

In [264]:
test1=pd.DataFrame(test)

In [265]:
test1.head(1)

,Science,Art
0,1,4


In [261]:
lens=test1.Science.str.len()

In [262]:
lens.max()

7.0